In [37]:
import os
import pandas as pd


In [38]:
file1 = "/Users/cla/Desktop/UM Data Science/Projects/Project - 2/ETL-Project/Data/MDC_Housing_Data.csv"
housing_data = pd.read_csv(file1)

In [39]:
file2 = "/Users/cla/Desktop/UM Data Science/Projects/Project - 2/ETL-Project/Data/ZIP_TRACT_2019.csv"
zip_vs_tract = pd.read_csv(file2)

In [40]:
file3 = '/Users/cla/Desktop/UM Data Science/Projects/Project - 2/ETL-Project/Data/ACSST5Y2018.S1902_data_with_overlays_2020-01-11T105425.csv'
income_data = pd.read_csv(file3)

In [41]:
housing_data
cols = [0,3,49]
housing_data = housing_data
housing_data = pd.DataFrame(housing_data.iloc[:, cols])


In [42]:
new = housing_data.drop(housing_data.index[[0,2]])
new

,"Miami-Dade County, FL - All Homes",Unnamed: 3,Unnamed: 49
1,Region Name,Current,Current
3,33010,"$321,100",---
4,33012,"$251,200","$1,807"
5,33013,"$341,000",---
6,33014,"$268,800","$1,892"
...,...,...,...
80,33190,"$267,800","$1,873"
81,33193,"$289,700","$1,980"
82,33194,"$368,400","$2,302"
83,33196,"$354,700","$2,208"


In [43]:
new_header = new.iloc[0] 
new = new[1:]
new.columns = new_header 


In [44]:
new.head(25)
col_names = ['zip_code','home_value','rent_value']

In [45]:
new.columns = col_names

In [46]:
home_value_byzip = new
home_value_byzip

,zip_code,home_value,rent_value
3,33010,"$321,100",---
4,33012,"$251,200","$1,807"
5,33013,"$341,000",---
6,33014,"$268,800","$1,892"
7,33015,"$271,400","$1,820"
...,...,...,...
80,33190,"$267,800","$1,873"
81,33193,"$289,700","$1,980"
82,33194,"$368,400","$2,302"
83,33196,"$354,700","$2,208"


In [47]:
zip_vs_tract = zip_vs_tract[['zip','tract']]

zip_vs_tract

,zip,tract
0,501,36103158607
1,601,72113071700
2,601,72001956800
3,601,72001956300
4,601,72001956600
...,...,...
170188,99925,2198000200
170189,99926,2198940100
170190,99927,2198000100
170191,99928,2130000100


In [48]:
zip_data = new.join(zip_vs_tract)
zip_data = zip_data[['zip_code','home_value','rent_value']]
zip_data['zip_code'] = zip_data.zip_code.astype(float)


In [49]:
zip_data

,zip_code,home_value,rent_value
3,33010.0,"$321,100",---
4,33012.0,"$251,200","$1,807"
5,33013.0,"$341,000",---
6,33014.0,"$268,800","$1,892"
7,33015.0,"$271,400","$1,820"
...,...,...,...
80,33190.0,"$267,800","$1,873"
81,33193.0,"$289,700","$1,980"
82,33194.0,"$368,400","$2,302"
83,33196.0,"$354,700","$2,208"


In [50]:
income_data = income_data[['GEO_ID','S1902_C03_001E']]

income_data

,GEO_ID,S1902_C03_001E
0,id,Estimate!!Mean income (dollars)!!HOUSEHOLD INC...
1,1400000US12086000107,80065
2,1400000US12086000109,51107
3,1400000US12086000113,96580
4,1400000US12086000115,155974
...,...,...
515,1400000US12086980700,12332
516,1400000US12086980800,-
517,1400000US12086980900,-
518,1400000US12086981000,-


In [51]:
income_data = income_data.drop(0)

In [52]:
# new data frame with split value columns 
new = income_data["GEO_ID"].str.split("US", n = 1, expand = True) 
  
# making separate first name column from new data frame 
income_data["x"]= new[0] 
  
# making separate last name column from new data frame 
income_data["Tract"]= new[1] 

In [53]:
income_data = income_data[['S1902_C03_001E','Tract']].rename(columns={'S1902_C03_001E':'Avg_income'})
income_data
income_data['Tract']=income_data['Tract'].astype(int)

In [54]:
#income_data = pd.merge(left = income_data,right = zip_vs_tract, left_on='Tract', right_on='tract')
merged_left = pd.merge(left=income_data,right=zip_vs_tract, how='left', left_on='Tract', right_on='tract')

In [55]:
merged_left = merged_left[['Avg_income','zip']]


In [56]:
merged_left

,Avg_income,zip
0,80065,33160.0
1,51107,33160.0
2,51107,33181.0
3,51107,33261.0
4,96580,33163.0
...,...,...
930,-,33185.0
931,-,33193.0
932,-,33132.0
933,-,33139.0


In [57]:
merged_left.dropna()
merged_left.Avg_income = pd.to_numeric(merged_left.Avg_income, errors='coerce').fillna(0).astype(int)


In [58]:
merged_left

,Avg_income,zip
0,80065,33160.0
1,51107,33160.0
2,51107,33181.0
3,51107,33261.0
4,96580,33163.0
...,...,...
930,0,33185.0
931,0,33193.0
932,0,33132.0
933,0,33139.0


In [59]:
avgincome_perzip = merged_left.groupby(['zip']).mean()


In [60]:
avgincome_perzip

,Avg_income
zip,
33002.0,55473.000000
33010.0,40904.846154
33011.0,32323.000000
33012.0,44724.055556
33013.0,44833.750000
...,...
33266.0,68208.000000
33269.0,56083.000000
33280.0,169910.000000


In [61]:
#income_homevalue = pd.merge(left=zip_data,right=merged_left, how='inner', left_on='zip_code', right_on='zip')
income_homevalue = pd.merge(left=avgincome_perzip,right=zip_data, how='inner', left_on='zip', right_on='zip_code')

In [62]:
income_homevalue['rule_of_thumb'] = (income_homevalue.Avg_income * 0.3 / 12).round(3)

In [63]:
income_homevalue['rent_value'] = income_homevalue['rent_value'].str.replace('$','').str.replace(',','')
income_homevalue['rent_value'] = pd.to_numeric(income_homevalue.rent_value, errors='coerce').fillna(0).astype(int)

In [64]:
income_homevalue['home_value'] = income_homevalue['home_value'].str.replace('$','').str.replace(',','')
income_homevalue['home_value'] = pd.to_numeric(income_homevalue.home_value, errors='coerce').fillna(0).astype(int)

In [65]:
income_homevalue['disparity'] = income_homevalue.rent_value - income_homevalue.rule_of_thumb

In [66]:
income_homevalue = income_homevalue.sort_values(by='disparity', ascending= False).sort_values(by='Avg_income')

In [67]:
income_homevalue = income_homevalue[income_homevalue.rent_value != 0]

In [68]:
top30_disparity = income_homevalue.head(30)

In [69]:
top30_disparity['zip_code'] = top30_disparity['zip_code'].astype(int)
top30_disparity['Avg_income'] = top30_disparity['Avg_income'].round(3)

/Users/cla/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/cla/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [70]:
top30_disparity = top30_disparity.reset_index(drop=True)
v

,Avg_income,zip_code,home_value,rent_value,rule_of_thumb,disparity
0,35514.667,33142,235600,1759,887.867,871.133
1,40237.200,33127,281800,1934,1005.930,928.070
2,40286.333,33147,244200,1760,1007.158,752.842
3,41506.000,33125,311700,1886,1037.650,848.350
4,42844.636,33136,243800,1889,1071.116,817.884
5,44724.056,33012,251200,1807,1118.101,688.899
6,46262.500,33167,250300,1811,1156.562,654.438
7,48374.417,33126,206500,1628,1209.360,418.640
8,48984.929,33150,246700,1838,1224.623,613.377
9,51047.462,33032,268900,1901,1276.187,624.813


In [73]:
zip_list = top30_disparity['zip_code'].tolist()

In [74]:
zip_list

[33142,
 33127,
 33147,
 33125,
 33136,
 33012,
 33167,
 33126,
 33150,
 33032,
 33130,
 33162,
 33034,
 33055,
 33056,
 33174,
 33172,
 33033,
 33168,
 33190,
 33169,
 33181,
 33030,
 33014,
 33193,
 33016,
 33161,
 33184,
 33194,
 33035]